In [1]:
import pandas as pd
import numpy as np
import plotly
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
import yfinance as yf
import dash
from dash import dcc
from dash import html


import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels.panel import compare
from linearmodels import RandomEffects
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import seaborn as sns

import math

c:\Users\jespe\anaconda3\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [19]:
df_econ = pd.read_excel('df_eco_ver1.xlsx', header=[0], index_col=0)

In [20]:
type = pd.Categorical(df_econ.port_id)
df_eco_multi = df_econ.set_index(['port_id', 'roll_id'])
df_eco_multi['groups'] = type

In [21]:
df_eco_copy = df_eco_multi.copy()

In [32]:
def Fe_reg_model(df, multi_df, dummies:True, summary:True):
    y_fe = df['Return']
    X_fe_e = df[['E_Score']]
    X_fe_s = df[['S_Score']]
    X_fe_g = df[['G_Score']]
    X_fe_all = df[['E_Score', 'S_Score', 'G_Score']]

    X_fe_e = sm.add_constant(X_fe_e)
    X_fe_s = sm.add_constant(X_fe_s)
    X_fe_g = sm.add_constant(X_fe_g)
    X_fe_all = sm.add_constant(X_fe_all)

    # Add entity fixed effects as dummy variables
    entity_dummies = pd.get_dummies(multi_df.index.get_level_values('port_id'), drop_first=True, prefix='port').astype(int)
    X_fe_e_dum = pd.concat([X_fe_e, entity_dummies], axis=1)
    X_fe_s_dum = pd.concat([X_fe_s, entity_dummies], axis=1)
    X_fe_g_dum = pd.concat([X_fe_g, entity_dummies], axis=1)
    X_fe_all_dum = pd.concat([X_fe_all, entity_dummies], axis=1)

    if dummies == True:
        # Estimate fixed effects model WIHT dummies
        fe_ret_e_dum = sm.OLS(y_fe, X_fe_e_dum).fit()
        fe_ret_s_dum = sm.OLS(y_fe, X_fe_s_dum).fit()
        fe_ret_g_dum = sm.OLS(y_fe, X_fe_g_dum).fit()
        fe_ret_all_dum = sm.OLS(y_fe, X_fe_all_dum).fit()
        if summary == True:
                return  fe_ret_e_dum.summary(), fe_ret_s_dum.summary(), fe_ret_g_dum.summary(), fe_ret_all_dum.summary()
        else:
                return  fe_ret_e_dum, fe_ret_s_dum, fe_ret_g_dum, fe_ret_all_dum
    else:
        print('SET "dummies = True, module not fully implemented')
        # #Estimate fixed effects model WITHOUT dummies - basically OLS and POOLED OLS
        # fe_ret_e = sm.OLS(y_fe, X_fe_e).fit()
        # fe_ret_s = sm.OLS(y_fe, X_fe_s).fit()
        # fe_ret_g = sm.OLS(y_fe, X_fe_g).fit()
        # fe_ret_all = sm.OLS(y_fe, X_fe_all).fit()
        # if summary == True:
        #         return  fe_ret_e.summary(), fe_ret_s.summary(), fe_ret_g.summary(), fe_ret_all.summary()
        # else:
        #         return  fe_ret_e, fe_ret_s, fe_ret_g, fe_ret_all


In [35]:
Fe_reg_model(df_econ, df_eco_copy, True, True)

(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                 Return   R-squared:                       0.296
 Model:                            OLS   Adj. R-squared:                  0.267
 Method:                 Least Squares   F-statistic:                     10.03
 Date:                Sun, 28 May 2023   Prob (F-statistic):           5.00e-08
 Time:                        22:35:09   Log-Likelihood:                 48.448
 No. Observations:                 125   AIC:                            -84.90
 Df Residuals:                     119   BIC:                            -67.93
 Df Model:                           5                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 ---------------------------------------------------------------------